In [269]:
import pandas as pd
import numpy as np
import random
import joblib
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, LabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report 
from sklearn.metrics import roc_curve, auc

In [270]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [271]:
#def sample_train_set(x, y, sample_num):
#    idx_list = random.sample(list(x.index), sample_num)
#    sampled_x = x.loc[idx_list]
#    sampled_y = y.loc[idx_list]
#    return np.array(sampled_x), np.array(sampled_y)

#### The Datasets

In [272]:
train_df = pd.read_csv('../../../anemia_ml4hc/data/train_set_basic.csv')
train_df = train_df.fillna(-1)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,14.728733,-1.000000,3.170892,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,44.186200,-1.0
1,10.405752,9.634615,5.659537,-1.000000,-1.000000,77.413788,212.671838,4.032519,0,0.887130,96.311597,-1.0,43.218595,-1.0,83.207518,31.217256,-1.0
2,15.132737,358.914888,1.842252,3.797487,315.102272,80.500314,-1.000000,5.639507,0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,45.398211,-1.0
3,11.340169,-1.000000,1.662209,2.441767,-1.000000,97.033963,102.079062,3.506041,1,1.020527,127.281715,-1.0,20.847013,-1.0,62.210273,34.020508,-1.0
4,6.691485,-1.000000,3.337971,-1.000000,-1.000000,99.838438,24.119564,2.010694,0,1.957666,34.633063,-1.0,34.612121,-1.0,112.411298,20.074456,-1.0


In [273]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000


#### Some useful functions and variables

In [274]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50400, 17), (14000, 17), (50400,), (14000,))

#### Decision Tree

In [275]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=constants.SEED).fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
test_df_dt = pd.DataFrame()
test_df_dt['y_actual'] = y_test
test_df_dt['y_pred'] = y_pred_dt
test_df_dt.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [276]:
success_rate_dt, success_df_dt = utils.success_rate(test_df_dt)
success_rate_dt

99.93571428571428

In [277]:
acc_dt, f1_dt, roc_auc_dt = utils.test(test_df_dt['y_actual'], test_df_dt['y_pred'])
acc_dt, f1_dt, roc_auc_dt

(99.93571428571428, 99.93659136698096, 99.96589206421822)

#### Random Forest Classifier

In [278]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=constants.SEED).fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
test_df_rf = pd.DataFrame()
test_df_rf['y_actual'] = y_test
test_df_rf['y_pred'] = y_pred_rf
test_df_rf.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [279]:
success_rate_rf, success_df_rf = utils.success_rate(test_df_rf)
success_rate_rf

99.90714285714286

In [280]:
acc_rf, f1_rf, roc_auc_rf = utils.test(test_df_rf['y_actual'], test_df_rf['y_pred'])
acc_rf, f1_rf, roc_auc_rf

(99.90714285714286, 99.90724000240444, 99.95046319909414)

#### XGBoost

In [281]:
# y_train_xgb = numerize_labels(y_train)
# y_test_xgb = numerize_labels(y_test)

In [282]:
import xgboost as xgb
xg = xgb.XGBClassifier(random_state=constants.SEED).fit(X_train, y_train)
y_pred_xg = xg.predict(X_test)
test_df_xg = pd.DataFrame()
test_df_xg['y_actual'] = y_test
test_df_xg['y_pred'] = y_pred_xg
test_df_xg.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [283]:
success_rate_xg, success_df_xg = utils.success_rate(test_df_xg)
success_rate_xg

99.9857142857143

In [284]:
acc_xg, f1_xg, roc_auc_xg = utils.test(test_df_xg['y_actual'], test_df_xg['y_pred'])
acc_xg, f1_xg, roc_auc_xg

(99.9857142857143, 99.98464035760173, 99.99201112442759)

#### Testing Rf and data with random zeros

In [268]:
# #data with random zeros
# X_train = np.loadtxt('data/zeros/X_train.txt', dtype=np.float32)
# #X_val = np.loadtxt('data/zeros/X_val.txt', dtype=np.float32)
# X_test = np.loadtxt('data/zeros/X_test.txt', dtype=np.float32)

# y_train = np.loadtxt('data/zeros/y_train.txt', dtype=int)
# #y_val = np.loadtxt('data/zeros/y_val.txt', dtype=int)
# y_test = np.loadtxt('data/zeros/y_test.txt', dtype=int)

In [91]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(random_state=42).fit(X_train, y_train)
# acc, f1, cr, cm, roc_auc, roc_auc2, y_pred  = test(rf, X_test, y_test) 
# print(f'Accuracy - {acc}, F1 Score Macro: {f1}, ROC AUC Score: {roc_auc}, ROC AUC 2: {roc_auc2}')
# print(f'Unique predicted classes: {np.unique(y_pred)}')